In [1]:
import os
os.environ["MODIN_ENGINE"] = "dask"
import modin.pandas as pd
# import pandas as pd
import numpy as np
import warnings
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import torch
nltk.download('punkt')

from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

#Inicializando modin
from distributed import Client
client = Client()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CCENTENO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
    Found GPU0 NVIDIA GeForce GTX 780 which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    


In [2]:
columns = ["PUBLICACION_CLEAN"]
df = pd.read_csv("you_comments.csv", sep="\t", engine="python", usecols=columns)
df= df[df['PUBLICACION_CLEAN'].notnull()]
df.rename(columns={'PUBLICACION_CLEAN':'PUBLICACION'}, inplace=True)
print(df.shape)
df.head(5)

(2590, 1)


,PUBLICACION
0,Exitosa Perú: La voz de los que no tienen voz ...
1,Vergüenza esta presentación.
2,Q VTV buenas
3,"Estás defiendo a un inepto corrupto, abogado P..."
4,Lo mejor seria marchar y que se vallan todos y...


In [3]:
columns1=["PUBLICACION"]
df1= pd.read_csv("PUBLICACIONES.csv", sep=",", usecols=columns1)
print(df1.shape)
df1.head(2)

(231295, 1)


,PUBLICACION
0,Muy interesante por lo gráficamente redactado....
1,@quintingrone @NairAliaga Nunca vio tanta gent...


In [7]:
#Dividir parrafo en oraciones teniendo en cuenta abreviaturas y otros
#====================================================================
dff= pd.concat([df[['PUBLICACION']],df1[['PUBLICACION']]], axis=0)

dff['oracion'] = dff.PUBLICACION.apply(lambda x: sent_tokenize(str(x), language='spanish'))
print(dff.shape)
dff.head(3)

(233885, 2)


,PUBLICACION,oracion
0,Exitosa Perú: La voz de los que no tienen voz ...,[Exitosa Perú: La voz de los que no tienen voz...
1,Vergüenza esta presentación.,[Vergüenza esta presentación.]
2,Q VTV buenas,[Q VTV buenas]


In [17]:
columns=['key','oracion']
dfo= pd.DataFrame(columns=columns)
for index, row in dff.iterrows():
    lista_oraciones = row['oracion']
    for oracion in lista_oraciones:
        # print(index, ':', oracion)
        diccionario = {'key':index, 'oracion': oracion}
        dfo = dfo.append(diccionario, ignore_index=True)

KeyboardInterrupt: 